In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib qt5

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io

from skimage.transform import hough_circle, hough_circle_peaks, hough_ellipse
from skimage.feature import canny
from skimage.draw import circle_perimeter, circle
from skimage.util import img_as_ubyte
from pathlib import Path
from skimage import data, color
plt.rcParams["figure.figsize"] = (20, 10)

In [3]:

path = Path('/home/pk/Documents/realtimeData/hetero40x/Pos103/phaseFast/img_000000017.tiff')

In [4]:
image = io.imread(path).astype('float32')

In [5]:
image.shape

(1488, 4096)

In [6]:
image.dtype

dtype('float32')

In [7]:
plt.imshow(image)

In [8]:
image = (image - np.mean(image))/np.std(image)

In [9]:
from skimage import filters

In [10]:
edges = canny(image, sigma=7)

In [11]:
plt.imshow(edges)

In [12]:
from scipy.ndimage import binary_fill_holes, binary_erosion

In [13]:
image = binary_fill_holes(edges)

In [15]:
plt.imshow(image)

In [16]:
image = binary_erosion(image)

In [17]:
plt.imshow(image)

In [31]:
def detect_circles(in_img):
    edges = canny(in_img, sigma=2)
    hough_radii = np.arange(10, , 2)
    hough_res = hough_circle(edges, hough_radii)
    accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=300)

    img1 = np.zeros(in_img.shape)
    img1 = color.gray2rgb(img1)
    for center_y, center_x, radius, (r, g, b, _) in zip(cy, cx, radii, 
                                          plt.cm.nipy_spectral(np.linspace(0,1, len(radii))) # color map
                                         ):
        circy, circx = circle(center_y, center_x, radius)
        img1[circy, circx] = (r*255, g*255, b*255)
    return img1


SyntaxError: invalid syntax (2565744971.py, line 3)

In [25]:
img = detect_circles(image)

/tmp/ipykernel_431803/2842679918.py:12: FutureWarning: `draw.circle` is deprecated in favor of `draw.disk`.`draw.circle` will be removed in version 0.19
  circy, circx = circle(center_y, center_x, radius)


In [27]:
plt.imshow(img)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [ ]:
radii

In [ ]:
cx

In [ ]:

cy